__BIOBSS - A python package for biological signal processing by OBSS__

_This package includes modules to process PPG, EDA and ACC signals recorded using wearable devices._


In [ ]:
#Import BIOBSS and the other required packages

import biobss
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#help(biobss)

This notebook includes guidelines to help using the package.

# Table of Contents
1.[Pipeline for formatting Empatica data files](#empatica)<br>
2.[Pipeline for formattion Polar Verity Sense data files](#polar)<br>
3.[Load the sample data](#sampledata)<br>
4.[PPG signal preprocessing pipeline](#ppg_pre)<br>
5.[PPG signal quality assessment pipeline](#ppg_sqa)<br>
6.[Respiratory rate estimation pipeline](#ppg_rr)<br>
7.[Calculation of PPG features](#ppg_features)<br>

### __Pipeline for formatting Empatica data files__
<a id="empatica"></a>
_If the data was recorded using an Empatica device, the methods below can be used to extract the required data files from the zip archive and correct for the time format._ 

In [ ]:
#Define the file paths for the zip files, temporary files and csv files (For Empatica file extraction and time-format correction)
zipfiles_dir="C:\\Users\\ipek.karakus\\Desktop\\biobss\\Empatica_files\\zip_files\\"
temp_dir="C:\\Users\\ipek.karakus\\Desktop\\biobss\\Empatica_files\\temp"
csvfiles_dir="C:\\Users\\ipek.karakus\\Desktop\\biobss\\Empatica_files\\csv_files"

#Define the name of the zip file (record id)
zip_file_name="1555678143_A01BC1.zip"

#This method first extracts the csv files from the zip archive to a temporary folder. Then, the files are renamed and moved into the final directory.
biobss.signaltools.e4_format.unzip_and_rename(zipfiles_dir, temp_dir, csvfiles_dir, zip_file_name)

#Define the filename and filesource for timestamp correction.
theid = '1555678143_A01BC1' #This is the subject ID number (name of file)
filesource = "C:\\Users\\ipek.karakus\\Desktop\\biobss\\Empatica_files\\csv_files\\" #This is the source folder that contains all of your participant folders

#Different methods are used to correct the timestamp since the content of each csv file depends on the type of signal. 
os.chdir(r'C:\Users\ipek.karakus\Desktop\biobss\Empatica_files\csv_files')
#Correct timestamp for EDA, TEMP, HR and BVP signal files.
listtyped = ['EDA','TEMP', 'HR','BVP'] 
[biobss.signaltools.e4_format.importandexport(filesource, theid, typed) for typed in listtyped]
#Correct timestamp for ACC signal files.
biobss.signaltools.e4_format.importandexportAcc(filesource, theid, 'ACC') 
#Correct timestamp for IBI signal files.
biobss.signaltools.e4_format.importandexportIBI(filesource, theid, 'IBI') 

### __Pipeline for formatting Polar Verity Sense data files__
<a id="polar"></a>
_If the data was recorded using a Polar device, the methods below can be used to save data files as csv file and and synchronize the signal from different sensors._ 

In [ ]:
#Define the file paths for the files
txt_dir="C:\\Users\\ipek.karakus\\Desktop\\biobss\\Polar_files\\txt_files\\"
csv_dir="C:\\Users\\ipek.karakus\\Desktop\\biobss\\Polar_files\\csv_files\\"
pkl_dir="C:\\Users\\ipek.karakus\\Desktop\\biobss\\Polar_files\\pkl_files\\"

#Txt files can be saved as csv files
biobss.reader.polar_format.txt_to_csv(txt_dir)

#Timestamps in iso format can be converted to milliseconds for a specific file. This can be done by referencing either to start time of the sensor or a given start time.
filepath=r'C:\Users\ipek.karakus\Desktop\biobss\Polar_files\csv_files\ik_d1_25042022_forearm_left\Polar_Sense_A40F6F28_20220425_113903_PPG.csv'
df= pd.read_csv(filepath)
time_msec=biobss.reader.polar_format.timestamp_to_msec(df['Phone timestamp']) 

#If required, the selected csv files can be updated by adding a 'Time_record (ms)' column, corresponds to time points in ms. The timepoints are calculated referenced to the earliest timestamp for all sensors.
csv_subdir="C:\\Users\\ipek.karakus\\Desktop\\biobss\\Polar_files\\csv_files\\ik_d1_25042022_forearm_left" #This time, the path should be defined for a specific record.
biobss.reader.update_csv(csv_dir=csv_subdir,marker=True)

#For synchronization of the signals, a common (overlapping region for all sensors) time array can be generated. This time, the path should be defined for a specific record.
time_list=biobss.reader.calculate_sync_time(csv_dir=csv_subdir,time_step=1,marker=True)

#If required, the time array can be saved as a txt file.
time_list=biobss.reader.calculate_sync_time(csv_dir=csv_subdir,time_step=1,marker=True,save_file=True)

#The signals can be synchronized by interpolating the signal for the given time_list.
data=biobss.reader.synchronize_signals(csv_subdir,time_list)

#Resampling can be applied if the required parameters (sampling_rate and resampling_rate are given) 
data=biobss.reader.synchronize_signals(csv_subdir,time_list,resampling_rate=100)

#If required, the synchronized signals can be saved as a csv file.
data=biobss.reader.synchronize_signals(csv_subdir,time_list,resampling_rate=100,save_files=True)

#The synchronized signals can be segmented for events
filepath=r'C:\Users\ipek.karakus\Desktop\biobss\Polar_files\csv_files\ik_d1_25042022_forearm_left\sync_ACC_PPG_MAGN_GYRO.csv'
markerpath=r'C:\Users\ipek.karakus\Desktop\biobss\Polar_files\csv_files\ik_d1_25042022_forearm_left\MARKER_20220425_114002.csv'
out_path=r'C:\Users\ipek.karakus\Desktop\biobss\Polar_files\csv_files\ik_d1_25042022_forearm_left\event_list.txt'
data=biobss.reader.segment_events(filepath,markerpath,['rest'],out_path)

#The event list can also be saved as txt file for later use
biobss.reader.segment_events(filepath,markerpath,['rest'],out_path,save_file=True)

#Csv file can be read into a dictionary
filepath=r'C:\Users\ipek.karakus\Desktop\biobss\Polar_files\csv_files\ik_d1_25042022_forearm_left\Polar_Sense_A40F6F28_20220425_113903_PPG.csv'
data=biobss.reader.polar_csv_reader(filepath,signal_type='PPG')

#Csv files can be saved as pkl files
biobss.reader.csv_to_pkl(csv_dir)

### __Load the sample data__
<a id="sampledata"></a>

In [ ]:
#Load the sample data

data_dir=r'C:\Users\ipek.karakus\Desktop\biobss\sample_data'

filename='ieee_wearable_P1_ppg.csv'

data=biobss.signaltools.load_csv(data_dir,filename)
sig=np.asarray(data.iloc[0,:])
fs=64
L=10


### __PPG Signal Preprocessing Pipeline__
<a id="ppg_pre"></a>

In [ ]:
#PPG signals can be filtered using the method below using pre-determined filter parameters

filtered_ppg=biobss.ppgtools.filter_ppg(sig,fs)

#or defining the parameters.

f_sig= biobss.signaltools.filter_signal(sig,'bandpass',2,fs,f1=0.5,f2=5)

#Signal peaks can be detected as below

info=biobss.signaltools.peakdetection.peak_detection(sig,fs,'peakdet',delta=0.01)

locs_peaks=info['Peak_locs']
peaks=info['Peaks']
locs_onsets=info['Trough_locs']
onsets=info['Troughs']

#and the results can be corrected considering the order of peaks and onsets
info=biobss.ppgtools.peak_control(locs_peaks,peaks,locs_onsets,onsets)

locs_peaks=info['Peak_locs']
peaks=info['Peaks']
locs_onsets=info['Trough_locs']
onsets=info['Troughs']

#If needed, first and second derivatives of the signal can be calculated using the method below.
info=biobss.signaltools.derivation.ppg_derivation(sig)
vpg_sig=info['VPG']
apg_sig=info['APG']

### __PPG Signal Quality Assessment Pipeline__
<a id="ppg_sqa"></a>

In [ ]:
#Clipping and flatline detection can be done using the methods below
#For clipping detection
info=biobss.ppgtools.detect_flatline_clipping(sig,1.0)
#For flatline detection
info=biobss.ppgtools.detect_flatline_clipping(sig,0.0001,10)

#The peaks should be detected for the following steps

info=biobss.signaltools.peakdetection.peak_detection(sig,fs,'peakdet',delta=0.01)
locs_peaks=info['Peak_locs']
peaks=info['Peaks']
locs_onsets=info['Trough_locs']
onsets=info['Troughs']

#and the results should be corrected considering the order of peaks and onsets
info=biobss.ppgtools.peak_control(locs_peaks,peaks,locs_onsets,onsets)
locs_peaks=info['Peak_locs']
peaks=info['Peaks']
locs_onsets=info['Trough_locs']
onsets=info['Troughs']

#Using the peak locations, the physiological and morphological limits can be checked

info=biobss.ppgtools.check_phys(locs_peaks,fs)
info=biobss.ppgtools.check_morph(locs_peaks,peaks,locs_onsets,onsets,fs)

#Template matching method can be applied using the method below.

info=biobss.ppgtools.template_matching(sig,locs_peaks)

### __Respiratory Rate Estimation Pipeline__
<a id="ppg_rr"></a>

In [ ]:
#For respiratory rate estimation, the ppg signal should be filtered before extracting respiratory signal
filt_sig=biobss.resptools.elim_vlf(sig,fs)
filt_sig=biobss.resptools.elim_vhf(sig,fs)

#Next, the PPG signal peaks and onsets are detected
info=biobss.signaltools.peakdetection.peak_detection(sig,fs,'peakdet',delta=0.01)
locs_peaks=info['Peak_locs']
peaks=info['Peaks']
locs_onsets=info['Trough_locs']
onsets=info['Troughs']

#In order to calculate the features correctly, the signal segment should start and end with an onset.  
#Additionally, there should be a single peak between consecutive onsets.

info=biobss.ppgtools.peak_control(locs_peaks,peaks,locs_onsets,onsets)
locs_peaks=info['Peak_locs']
peaks=info['Peaks']
locs_onsets=info['Trough_locs']
onsets=info['Troughs']

#The next step is to extract the respiratory signal from the PPG signal.

info=biobss.resptools.extract_resp_sig(locs_peaks,peaks,onsets,fs,['AM','FM','BW'],10)

#The method above extracts the respiratory signals based on three modulation types seperately. To select the signal for a specific method.

y_am=info['am_y']
x_am=info['am_x']
y_fm=info['fm_y']
x_fm=info['fm_x']
y_bw=info['bw_y']
x_bw=info['bw_x']

#If required, the extracted respiratory signal can be filtered prior to respiratory rate estimation.

info=biobss.resptools.filter_resp_sig(resampling_rate=10, am_sig=y_am, am_x=x_am, fm_sig=y_fm, fm_x=x_fm, bw_sig=y_bw, bw_x=x_bw)

y_am=info['am_sig']
x_am=info['am_x']
y_fm=info['fm_sig']
x_fm=info['fm_x']
y_bw=info['bw_sig']
x_bw=info['bw_x']

#If required, the respiratory quality assessment can be applied. Autocorrelation or Hjorth approaches can be used. 
#Note that, this step should be applied for a specific modulation type.

rqi_am=biobss.resptools.calc_rqi(y_am,resampling_rate=10)
rqi_fm=biobss.resptools.calc_rqi(y_fm,resampling_rate=10)
rqi_bw=biobss.resptools.calc_rqi(y_bw,resampling_rate=10)

#Now, the respiratory rate can be estimated. Note that, this method should be used for a specific modulation type.

rr_am=biobss.resptools.estimate_rr(y_am,10,method='peakdet')
rr_fm=biobss.resptools.estimate_rr(y_fm,10,method='peakdet')
rr_bw=biobss.resptools.estimate_rr(y_bw,10,method='peakdet')

#Seperate respiratory rates can be fused into a single value. SmartFusion method does not take RQIs into consideration.

rr_fused=biobss.resptools.fuse_rr(fusion_method='SmartFusion', rqi=None, rr_am=rr_am, rr_fm=rr_fm, rr_bw=rr_bw)

#In order to calculate the weighted average for fusion of RRs, the fusion_method should be selected as 'QualityFusion'.
#RQIs (array) should be provided as a keyworded argument. The order of elements in the array should match with the order RRs of different modulation types.
rqi=[rqi_am['hjorth'],rqi_fm['hjorth'],rqi_bw['hjorth']]
rr_fused=biobss.resptools.fuse_rr(fusion_method='QualityFusion', rqi=rqi, rr_am=rr_am, rr_fm=rr_fm, rr_bw=rr_bw)

### __Calculation of PPG features__
<a id="ppg_features"></a>

In [ ]:
#Cycle-based features can be calculated
ppg_cycle=biobss.ppgtools.extract_features.from_cycles(sig,locs_peaks,peaks,locs_onsets,onsets,fs,feature_types=['Time','Stat'],prefix='ppg')

#Segment-based features can be calculated
ppg_segment=biobss.ppgtools.extract_features.from_segment(sig,fs,feature_types=['Time','Stat','Freq'], prefix='ppg')

